# PyTorch 从零实现 Transformer

## 目录

1. Introduction
2. Basic Components
    1. Create Word Embeddings
    2. Positional Encoding
    3. Self Attention
3. Encoder
4. Decoder
5. Testing
6. Some useful resources

# 1.Introduction

本内容如何使用 Pytorch 从零开始实现 Transformer 架构，想要深入了解 Transformer 原理推荐下面内容：

1. [我自己写的笔记](https://github.com/CliffKai/DeepL-by-Pytorch/blob/master/Paper%20Reading/NLP/Transformer.md)
2. [Transformer 论文:Attention is All you Need](https://proceedings.neurips.cc/paper/2017/hash/3f5ee243547dee91fbd053c1c4a845aa-Abstract.html)
3. [李沐老师的d2l](https://zh-v2.d2l.ai/chapter_attention-mechanisms/index.html)
4. [Jay Alammar 的 Blog](https://jalammar.github.io/illustrated-transformer/)

![Figure_1](../../images/Transformer_from_scratch_Figure_1.png)

# 2.Basic Components

首先我们导入所需的库

In [4]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import torchtext
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
print(torch.__version__)
print(torch.cuda.is_available())

OSError: [WinError 127] 找不到指定的程序。

## 2.1 Create Word Embeddings

首先我们需要将输入序列中的每个词转化为一个嵌入向量，将离散的词语表示转换为具有语义意义的稠密向量表示，以便神经网络可以理解、处理和学习语言中的词语之间的关系。

这里我们假设每个嵌入向量的维度是 512，构建一个大小为 100 的词汇表，嵌入矩阵为 100 × 512。